# Fitting psychometric functions

This notebook analyzes data from `Pilot 11` dataset.

In [1]:
library(data.table)
library(ggplot2)        

## Dataset handling

Let's load and inspect the dataset.

In [2]:
# load dataset
data <- fread('../../data/Pilot11/psychophysical_data.csv')

In [3]:
str(data)

Classes ‘data.table’ and 'data.frame':	476 obs. of  9 variables:
 $ validTrialCount: int  1 2 3 4 5 6 7 8 9 10 ...
 $ choice         : chr  "left" "left" "left" "left" ...
 $ correct        : logi  TRUE TRUE TRUE FALSE TRUE FALSE ...
 $ presenceCP     : chr  "no" "yes" "no" "yes" ...
 $ viewingDuration: num  0.5 0.4 0.2 0.5 0.4 0.5 0.4 0.5 0.1 0.3 ...
 $ signedCoherence: int  -60 NA -60 NA NA NA 12 12 -12 0 ...
 $ coherence      : int  60 24 60 12 12 12 12 12 12 0 ...
 $ choice_time    : num  0.921 0.86 1.125 0.875 0.979 ...
 $ exactVD        : num  0.525 0.424 0.203 0.525 0.407 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [4]:
head(data)

validTrialCount,choice,correct,presenceCP,viewingDuration,signedCoherence,coherence,choice_time,exactVD
1,left,TRUE,no,0.5,-60,60,0.9206421,0.5248945
2,left,TRUE,yes,0.4,NA,24,0.8598706,0.4237288
3,left,TRUE,no,0.2,-60,60,1.1245399,0.2033898
4,left,FALSE,yes,0.5,NA,12,0.8751249,0.5254237
5,left,TRUE,yes,0.4,NA,12,0.9794417,0.4067797
6,left,FALSE,yes,0.5,NA,12,1.1367451,0.5073636


## With `quickpsy` package

In [ ]:
# for an explanation of masked objects, see here:
# https://stackoverflow.com/a/39137111
library(quickpsy)

In [ ]:
# let's recap the tables in the environment up until now
tables()

In [ ]:
str(TRIALS[,.(signedCoherence, presenceCP, choice, viewingDuration)])

In [ ]:
str(TRIALS[,`:=`(presenceCP=as.factor(presenceCP), 
                viewingDuration=as.factor(viewingDuration))][,
                                                             .(signedCoherence, presenceCP, choice, viewingDuration)])

In [ ]:
a <- factor(c(3,4,5))
a
a[2] == '4'

In [ ]:
# let's set presenceCP and viewingDuration as factor variables in a temporary data.table before upcoming grouping
tmp <- TRIALS[,`:=`(presenceCP=as.factor(presenceCP), viewingDuration=as.factor(viewingDuration))]

# build data.table used by the quickpsy package
psychometric <- tmp[!is.na(validTrialCount), 
                       .(numChooseRight=sum(choice == "1"), N=.N), 
                       by=.(signedCoherence, presenceCP, viewingDuration)]
str(psychometric)

In [ ]:
unique(psychometric$signedCoherence)

In [ ]:
# perform the fits
fit <- quickpsy(psychometric, signedCoherence, numChooseRight, N, grouping = .(presenceCP, viewingDuration),
               log=FALSE, fun=logistic_fun, lapses=TRUE)

In [ ]:
str(fit)

In [ ]:
plot(fit) + xlim(c(-60,60))

In [ ]:
plotpar(fit) + ylim(c(0,.25))

In [ ]:
plotthresholds(fit) + ylim(c(-10,10))


## With `glm` function

In [ ]:
psychometric[,numChooseLeft := N - numChooseRight]
str(psychometric)

In [ ]:
# let's first follow this example:
# http://www.dlinares.org/psychopract.html#fitting-using-generalized-linear-models
model <- glm( cbind(numChooseRight, numChooseLeft) ~ signedCoherence, 
             data= psychometric, 
             family = binomial(probit))

xseq <- seq(-60, 60, 1)
yseq <- predict(model, data.frame(signedCoherence = xseq), type = 'response')
curve <- data.frame(xseq, yseq)

p <- ggplot() +
  geom_point(data = psychometric[,.(signedCoherence, chooseRight=numChooseRight / N)], 
             aes(x = signedCoherence, y = chooseRight)) +
  geom_line(data = curve,aes(x = xseq, y = yseq))
p

The problem here is that I have a single regressor, namely, `signedCoherence`. This is not what I want.

In [ ]:
# Try instead this example
#https://www.statmethods.net/advstats/glm.html
# with good interpretation here:
# https://stats.idre.ucla.edu/r/dae/logit-regression/
tmp_glm <- TRIALS[!is.na(validTrialCount)]

tmp_glm[,choice:=droplevels(choice)]        # drop unused level "NA" for choice variable
levels(tmp_glm$choice) <- c('left','right') # rename remaining levels

# treat presenceCP as factor and rename the labels
tmp_glm[,presenceCP:=as.factor(presenceCP)]
levels(tmp_glm$presenceCP) <- c('no-CP','CP')

str(tmp_glm[,.(choice, presenceCP, viewingDuration, signedCoherence)])

In [ ]:
fit2 <- glm(choice ~ signedCoherence + viewingDuration + presenceCP, data = tmp_glm, family=binomial())

In [ ]:
summary(fit2)

In [ ]:
confint(fit2) # 95% CI for the coefficients, using profiled log-likelihood
confint.default(fit2)  # CIs using standard errors

In [ ]:
# odds ratios and 95% CI
exp(cbind(OddsRatio = coef(fit2), confint(fit2)))

In [ ]:
library("aod")
wald.test(b = coef(fit2), Sigma = vcov(fit2), Terms = 4)

In [ ]:
# better to use this
# https://stats.idre.ucla.edu/r/dae/logit-regression/
predict(fit2, type="response") # predicted values
residuals(fit2, type="deviance") # residuals

In [ ]:
cdplot(choice~signedCoherence, data=tmp_glm) 

In [ ]:
cdplot(choice~viewingDuration, data=tmp_glm) 

In [ ]:
# now try this
#https://datascienceplus.com/perform-logistic-regression-in-r/